In [0]:
%tensorflow_version 1.15

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [0]:
from __future__ import print_function
from functools import reduce
import json
import os
import re
import tarfile
import tempfile
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import merge, recurrent, Dense, Input, Dropout, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2
from keras.utils import np_utils
import os, re, csv, math, codecs
from keras.layers import concatenate, Flatten, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Activation, Input, Concatenate,Dot,RepeatVector,TimeDistributed,Multiply,Lambda,Bidirectional, LSTM, Reshape
import keras.backend as K
from keras.activations import softmax
from keras.models import Model
from keras import optimizers
from keras.activations import softmax
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
np.random.seed(1337)  # for reproducibility

Using TensorFlow backend.


In [0]:
#%tensorflow_version 1.15

In [0]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip

--2020-05-15 10:09:17--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  29.4MB/s    in 3.1s    

2020-05-15 10:09:20 (29.4 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]



In [0]:
!unzip snli_1.0.zip

Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_test.txt  
  inflating: snli_1.0/snli_1.0_train.jsonl  
  inflating: snli_1.0/snli_1.0_train.txt  
  inflating: __MACOSX/._snli_1.0     


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-05-15 10:09:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-15 10:09:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-15 10:09:34--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


###Funciones

In [0]:
def extract_tokens_from_binary_parse(parse):
    return parse.replace('(', ' ').replace(')', ' ').replace('-LRB-', '(').replace('-RRB-', ')').split()

In [0]:
def get_data(fn, limit=None):
  raw_data = list(yield_examples(fn=fn, limit=limit))
  left = [s1 for _, s1, s2 in raw_data]
  right = [s2 for _, s1, s2 in raw_data]
  print(max(len(x.split()) for x in left))
  print(max(len(x.split()) for x in right))

  LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
  Y = np.array([LABELS[l] for l, s1, s2 in raw_data])
  Y = np_utils.to_categorical(Y, len(LABELS))

  return left, right, Y

In [0]:
def yield_examples(fn, skip_no_majority=True, limit=None):
  for i, line in enumerate(open(fn)):
    if limit and i > limit:
      break
    data = json.loads(line)
    label = data['gold_label']
    s1 = ' '.join(extract_tokens_from_binary_parse(data['sentence1_binary_parse']))
    s2 = ' '.join(extract_tokens_from_binary_parse(data['sentence2_binary_parse']))
    if skip_no_majority and label == '-':
      continue
    yield (label, s1, s2)

###Prepare data

In [0]:
to_seq = lambda X: pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_LEN)
prepare_data = lambda data: (to_seq(data[0]), to_seq(data[1]), data[2])

###Import Data

In [0]:
training = get_data('/content/snli_1.0/snli_1.0_train.jsonl')
validation = get_data('/content/snli_1.0/snli_1.0_dev.jsonl')
test = get_data('/content/snli_1.0/snli_1.0_test.jsonl')

82
62
59
55
57
30


In [0]:
#tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token="UNK")
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(training[0] + training[1])

In [0]:
# Lowest index from the tokenizer is 1 - we need to include 0 in our vocab count
VOCAB = len(tokenizer.word_counts) + 1
LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}

### Veamos algunos ejemplos


In [0]:
training[0][0]

'A person on a horse jumps over a broken down airplane .'

In [0]:
training[1][0]

'A person is training his horse for a competition .'

In [0]:
training[2][0]

array([0., 1., 0.], dtype=float32)

###Prepare dataset - Training, Validation, Test

In [0]:
MAX_LEN = 42
training = prepare_data(training)
validation = prepare_data(validation)
test = prepare_data(test)

In [0]:
  #Ejemplo de prepare

In [0]:
training[0][0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    3,   45,    8,
          2,  193,  205,   81,    2, 1171,   40,  822,    1], dtype=int32)

In [0]:
training[1][0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    3,
         45,    5, 1175,   21,  193,   38,    2,  456,    1], dtype=int32)

In [0]:
training[2][0]

array([0., 1., 0.], dtype=float32)

In [0]:
to_seq = lambda X: pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_LEN)

In [0]:
prepare_data = lambda data: (to_seq(data[0]), to_seq(data[1]), data[2])

In [0]:
print('Build model...')
print('Vocab size =', VOCAB)

Build model...
Vocab size = 42391


###Load Embeddings

In [0]:
#load embeddings
#EMBEDDING_DIR = "/content/drive/My Drive/"
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('/content/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...
found 400000 word vectors


In [0]:
EMBED_HIDDEN_SIZE = 300
# Embedding layer
embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable=False)
# A dense layer applied over each sequence point
translate = TimeDistributed(Dense(SENT_HIDDEN_SIZE, activation='relu'))
# A layer to sum up the sequence of words
SumEmbeddings = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=(SENT_HIDDEN_SIZE, ))

###Prepare embedding matrix

In [0]:
# prepare embedding matrix
embed_dim=100
embedding_matrix=np.zeros([VOCAB+4,embed_dim])
for word, idx in tokenizer.word_index.items():
  if idx <= VOCAB and word in embeddings_index:
    embedding_matrix[idx+3,:]=embeddings_index[word]

###Translate

In [0]:
SENT_HIDDEN_SIZE = 300
translate = TimeDistributed(Dense(SENT_HIDDEN_SIZE, activation='relu'))

###Premisas e Hipotesis

In [0]:
VOCAB=VOCAB+4
premise = Input(shape=(MAX_LEN,), dtype='int32')
hypothesis = Input(shape=(MAX_LEN,), dtype='int32')

prem = embed(premise)
hypo = embed(hypothesis)

prem = translate(prem)
hypo = translate(hypo)

In [0]:
value_dim=100

In [0]:
training[0].shape

(549367, 42)

###Inputs and concatenate

In [0]:
DP = 0.1
RNN = recurrent.LSTM
# RNN = lambda *args, **kwargs: Bidirectional(recurrent.LSTM(*args, **kwargs))
rnn_kwargs = dict(output_dim=SENT_HIDDEN_SIZE, dropout_W=DP, dropout_U=DP)
rnn = RNN(return_sequences=False, **rnn_kwargs)
#rnn_kwargs = dict(output_dim=SENT_HIDDEN_SIZE, dropout_W=DP, dropout_U=DP)
VOCAB=VOCAB+4
# 2 pairs of input sentences
premise = Input(shape=(MAX_LEN, ), dtype='int32',)
hypothesis = Input(shape=(MAX_LEN, ), dtype='int32')
# Get the word embeddings for each of these 2 pairs
prem = embed(premise)  # [batchsize, Psize, Embedsize]
hypo = embed(hypothesis) # [batchsize, Hsize, Embedsize]
# Apply the Dense layer
prem = translate(prem)
hypo = translate(hypo)
# Sum up the sequence
prem = rnn(prem)
hypo = rnn(hypo)
prem = BatchNormalization()(prem)
hypo = BatchNormalization()(hypo)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=False, units=300, dropout=0.1, recurrent_dropout=0.1)`
  """


In [0]:
# Combined the 2 sentences
joint = concatenate([prem, hypo])
joint = Dropout(0.4)(joint)

###Capas Densas

In [0]:
L2 = 4e-6
for i in range(3):
        joint = Dense(2 * SENT_HIDDEN_SIZE, activation='relu', kernel_regularizer=l2(L2) if L2 else None)(joint)
        joint = Dropout(0.4)(joint)



###Softmax

In [0]:
pred = Dense(len(LABELS), activation='softmax')(joint)

###Model

In [0]:
model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 42)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 42)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 42, 300)      12717300    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 42, 300)      90300       embedding_1[2][0]          

In [0]:
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
_, tmpfn = tempfile.mkstemp()
BATCH_SIZE = 512
MAX_EPOCHS = 5
# Save the best model during validation and bail out of training early if we're not improving
callbacks = [#EarlyStopping(patience=PATIENCE),
        ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True),
        History(),
    ]
#model.fit(
 #       [training[0], training[1]],
 #       training[2],
  #      batch_size=BATCH_SIZE,
  #      epochs=MAX_EPOCHS,
  #      validation_data=([validation[0], validation[1]], validation[2]),
   #     callbacks=callbacks)

In [0]:
model.save("model.h5")

In [0]:
from keras.models import load_model

In [0]:
model = load_model('model.h5')

###Evaluate

In [0]:
# Restore the best found model during validation
#model.load_weights(tmpfn)

loss, acc = model.evaluate([test[0], test[1]], test[2], batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

9824/9824 [==============================] - 29s 3ms/step
Test loss / test accuracy = 0.6884 / 0.7211


In [0]:
predict = model.predict([test[0], test[1]])

In [0]:
predict

array([[0.3099998 , 0.4193015 , 0.2706987 ],
       [0.19894674, 0.36704957, 0.4340037 ],
       [0.28794542, 0.25216246, 0.45989218],
       ...,
       [0.9822329 , 0.01034008, 0.00742698],
       [0.07046628, 0.1640845 , 0.76544917],
       [0.18254942, 0.50951105, 0.3079395 ]], dtype=float32)

In [0]:
y_classes = predict.argmax(axis=-1)

In [0]:
len
len(y_classes)

9824